In [2]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [3]:
import os

os.chdir("..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/datasource_hackathon


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
from sklearn.model_selection import train_test_split
import warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common import space
warnings.filterwarnings("ignore")

In [5]:
all_feats_df = pd.read_pickle("engineered_data/merged_data_engineered_all_features_weekly.pkl")

In [6]:
all_feats_df.shape

(1615437, 86)

In [7]:
all_feats_df = all_feats_df.drop(
    columns=["Price", "Inventory"], axis=1, errors="ignore"
)
all_feats_df.replace([np.inf, -np.inf], 1e-6, inplace=True)

In [8]:
all_feats_df[
    [
        "Client",
        "Warehouse",
        "Product",
        "monthly_inventory_category",
        "weekly_inventory_category",
        "monthly_period_inventory_category",
        "demand_category",
    ]
] = all_feats_df[
    [
        "Client",
        "Warehouse",
        "Product",
        "monthly_inventory_category",
        "weekly_inventory_category",
        "monthly_period_inventory_category",
        "demand_category",
    ]
].astype(
    "object"
)

In [9]:
val_dt = all_feats_df.loc[all_feats_df["Tag"] == "Train", "Date"].max() - pd.DateOffset(
    weeks=3
)
val_dt = str(val_dt.date())

In [10]:
train_all_feats_df = all_feats_df.loc[all_feats_df["Tag"] == "Train"].copy()
train_df = train_all_feats_df.loc[train_all_feats_df["Date"] < val_dt].copy()
val_df = train_all_feats_df.loc[train_all_feats_df["Date"] >= val_dt].copy()
test_all_feats_df = all_feats_df.loc[all_feats_df["Tag"] == "Test"].copy()

In [11]:
train_df.shape, val_df.shape, test_all_feats_df.shape

((1360090, 84), (59658, 84), (195689, 84))

In [12]:
train_df = train_df.set_index(["id", "Date"])
val_df = val_df.set_index(["id", "Date"])
test_all_feats_df = test_all_feats_df.set_index(["id", "Date"])

In [13]:
keep_cols = [
    "Client",
    "Warehouse",
    "Product",
    "monthly_inventory_category",
    "weekly_inventory_category",
    "monthly_period_inventory_category",
    "demand_category",
    "Sales",
    "Sales_lag_13",
    "Sales_roll_mean_3_lag_13",
    "Sales_roll_min_3_lag_13",
    "Sales_roll_max_3_lag_13",
    "Sales_roll_median_3_lag_13",
    "Inventory_change_rate_lag13",
    "Inventory_avg_last_4_weeks_lag13",
    "Month",
    "WeekOfYear",
    "DayOfMonth",
    "Quarter",
    "Month_sin",
    "Month_cos",
    "WeekOfYear_sin",
    "WeekOfYear_cos",
    "month_period",
]

keep_cols = train_df.columns.tolist()

In [14]:
label = 'Sales'

In [15]:
train_df = train_df[keep_cols]
y_val = val_df[label]
val_df_nolabel = val_df.drop(columns=[label])
test_all_feats_df = test_all_feats_df[[col for col in keep_cols if col != label]]

In [18]:
train_df.shape, val_df.shape, test_all_feats_df.shape, val_df_nolabel.shape

((1360090, 82), (59658, 82), (195689, 81), (59658, 81))

In [17]:
train_df.head()

Client Warehouse Product  Sales    Tag  Sales_lag_13  \
id          Date                                                              
0_100_10705 2020-07-06      0       100   10705    7.0  Train      6.999956   
            2020-07-13      0       100   10705    7.0  Train      7.001150   
            2020-07-20      0       100   10705    7.0  Train      6.996373   
            2020-07-27      0       100   10705    7.0  Train      7.000848   
            2020-08-03      0       100   10705    7.0  Train      7.002479   

                        Sales_lag_14  Sales_lag_15  Sales_lag_16  \
id          Date                                                   
0_100_10705 2020-07-06      6.999588      6.999479      7.001045   
            2020-07-13      6.998959      6.999008      6.999520   
            2020-07-20      6.999088      7.000560      7.000849   
            2020-07-27      6.998955      6.999397      6.999729   
            2020-08-03      6.999690      6.999445      7.000737   

                        Price_lag_13  Price_lag_14  Price_lag_15  \
id          Date                                                   
0_100_10705 2020-07-06     17.288617     17.287732     17.290314   
            2020-07-13     17.289693     17.289902     17.289309   
            2020-07-20     17.289971     17.288611     17.290257   
            2020-07-27     17.290186     17.289502     17.288695   
            2020-08-03     17.290153     17.289567     17.289511   

                        Price_lag_16  Sales_lag_14_diff  Sales_lag_15_diff  \
id          Date                                                             
0_100_10705 2020-07-06     17.290044          -0.000369          -0.000108   
            2020-07-13     17.288605          -0.002191           0.000049   
            2020-07-20     17.288971           0.002715           0.001472   
            2020-07-27     17.290163          -0.001893           0.000442   
            2020-08-03     17.289458          -0.002789          -0.000245   

                        Sales_lag_16_diff  Price_lag_14_diff  \
id          Date                                               
0_100_10705 2020-07-06           0.001566          -0.000885   
            2020-07-13           0.000511           0.000209   
            2020-07-20           0.000289          -0.001360   
            2020-07-27           0.000332          -0.000684   
            2020-08-03           0.001292          -0.000586   

                        Price_lag_15_diff  Price_lag_16_diff  \
id          Date                                               
0_100_10705 2020-07-06           0.002582          -0.000271   
            2020-07-13          -0.000593          -0.000704   
            2020-07-20           0.001646          -0.001287   
            2020-07-27          -0.000807           0.001468   
            2020-08-03          -0.000057          -0.000052   

                        Sales_roll_mean_3_lag_13  Sales_roll_min_3_lag_13  \
id          Date                                                            
0_100_10705 2020-07-06                  7.000073                 7.000751   
            2020-07-13                  6.999984                 6.998007   
            2020-07-20                  7.000520                 6.998148   
            2020-07-27                  7.000033                 7.000362   
            2020-08-03                  7.000936                 6.998244   

                        Sales_roll_max_3_lag_13  Sales_roll_median_3_lag_13  \
id          Date                                                              
0_100_10705 2020-07-06                 6.999460                    6.999297   
            2020-07-13                 7.001240                    7.000906   
            2020-07-20                 7.002468                    6.998770   
            2020-07-27                 6.999722                    6.999971   
            2020-08-03                 6.998990         

In [31]:
# all_feats_df_train["Sales"] = np.log(all_feats_df_train["Sales"] + 1e-6)
# all_feats_df_test["Sales"] = np.log1p(all_feats_df_test["Sales"] + 1e-6)

In [19]:
metric='smape'

In [20]:
gbm_options = (
    {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
        "num_boost_round": space.Int(lower=100, upper=1000, default=200),
        "num_leaves": space.Int(lower=26, upper=66, default=36),
        "learning_rate": space.Real(0.001, 0.1, log=True),
    }
)
xgb_options = (
    {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
        "max_depth": space.Int(lower=6, upper=15, default=10),
        "eta": space.Real(0.001, 0.1, log=True),
    }
)
cb_options = {  # specifies non-default hyperparameter values for neural network models
    "depth": space.Int(lower=1, upper=12, default=5),
    "learning_rate": space.Real(0.001, 0.1, log=True),
}

In [21]:
hyperparameters = {  # hyperparameters of each model type
    "GBM": gbm_options,
    "XGB": xgb_options,
    #                     "CAT":{},
}
num_trials = (
    10  # try at most 5 different hyperparameter configurations for each type of model
)
search_strategy = (
    "auto"  # to tune hyperparameters using random search routine with a local scheduler
)

hyperparameter_tune_kwargs = (
    {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
        "num_trials": num_trials,
        "scheduler": "local",
        "searcher": search_strategy,
    }
)  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    # presets="high_quality",
    refit_full=True,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240917_110444"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #73~20.04.1-Ubuntu SMP Mon May 6 09:43:44 UTC 2024
CPU Count:          16
Memory Avail:       96.38 GB / 108.07 GB (89.2%)
Disk Space Avail:   5004.61 GB / 5120.00 GB (97.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality'

  0%|          | 0/10 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	-0.7692	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	15.16s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-0.6417	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	15.1s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-0.6394	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	21.84s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-0.6385	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	9.79s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-0.6406	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	35.6s	 = Training   runtime
	0.09s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	-0.7048	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	40.06s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: Light

  0%|          | 0/10 [00:00<?, ?it/s]

Fitted model: XGBoost/T1 ...
	-0.642	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	24.6s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: XGBoost/T2 ...
	-0.6448	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	29.16s	 = Training   runtime
	0.12s	 = Validation runtime
Fitted model: XGBoost/T3 ...
	-0.6345	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	94.24s	 = Training   runtime
	0.27s	 = Validation runtime
Fitted model: XGBoost/T4 ...
	-0.6352	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	28.39s	 = Training   runtime
	0.21s	 = Validation runtime
Fitted model: XGBoost/T5 ...
	-0.6446	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	99.28s	 = Training   runtime
	0.21s	 = Validation runtime
Fitted model: XGBoost/T6 ...
	-0.6475	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	21.58s	 = Training   runtime
	0.12s	 = Validation runtime
Fitted model: XGBoost/T7

In [22]:
y_pred = predictor.predict(val_df_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(val_df, auxiliary_metrics=False)

Predictions:   [4.262292861938477, 4.0274248123168945, 4.0274248123168945, 4.0163373947143555, 10.882429122924805]


In [23]:
predictor.leaderboard(val_df)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.694346,-0.634016,symmetric_mean_absolute_percentage_error,2.072302,0.502753,132.467609,0.027146,0.000884,0.047704,2,True,21
1,XGBoost/T4_FULL,-0.694527,NaN,symmetric_mean_absolute_percentage_error,0.315337,NaN,21.978012,0.315337,NaN,21.978012,1,True,35
2,XGBoost/T3,-0.694636,-0.634527,symmetric_mean_absolute_percentage_error,1.518456,0.273689,94.243949,1.518456,0.273689,94.243949,1,True,13
3,WeightedEnsemble_L2_FULL,-0.694652,NaN,symmetric_mean_absolute_percentage_error,1.421892,NaN,92.537081,0.027320,NaN,0.047704,2,True,42
4,XGBoost/T3_FULL,-0.694996,NaN,symmetric_mean_absolute_percentage_error,1.006590,NaN,62.802694,1.006590,NaN,62.802694,1,True,34
5,XGBoost/T4,-0.695191,-0.635155,symmetric_mean_absolute_percentage_error,0.391617,0.205624,28.389438,0.391617,0.205624,28.389438,1,True,14
6,XGBoost/T7,-0.695424,-0.636981,symmetric_mean_absolute_percentage_error,0.350533,0.113340,29.235090,0.350533,0.113340,29.235090,1,True,17
7,LightGBM/T4,-0.695695,-0.638484,symmetric_mean_absolute_percentage_error,0.135083,0.022557,9.786518,0.135083,0.022557,9.786518,1,True,4
8,XGBoost/T7_FULL,-0.695787,NaN,symmetric_mean_absolute_percentage_error,0.356521,NaN,21.690835,0.356521,NaN,21.690835,1,True,38
9,LightGBM/T3,-0.696323,-0.639441,symmetric_mean_absolute_percentage_error,0.186528,0.060976,21.844181,0.186528,0.060976,21.844181,1,True,3


In [24]:
feats_df = predictor.feature_importance(
    train_df,
    subsample_size=int(len(train_df) / 3),
    num_shuffle_sets=3,
)

These features in provided data are not utilized by the predictor and will be ignored: ['Tag', 'WeekOfYear', 'WeekOfYear_sin', 'WeekOfYear_cos']
Computing feature importance via permutation shuffling for 77 features using 453363 rows with 3 shuffle sets...
	1942.8s	= Expected runtime (647.6s per shuffle set)
	1749.68s	= Actual runtime (Completed 3 of 3 shuffle sets)


In [54]:
feats_df["cumsum"] = (feats_df["importance"] / feats_df["importance"].sum()).cumsum()

In [58]:
feats_df.loc[feats_df["cumsum"] <= 0.9].index.tolist()

['Sales_roll_mean_6_lag_13',
 'demand_category',
 'Sales_roll_median_6_lag_13',
 'Sales_ewm_alpha_08_lag_13',
 'Client',
 'Sales_lag_16',
 'Warehouse',
 'Sales_roll_max_6_lag_13',
 'Sales_ewm_alpha_09_lag_13',
 'Sales_lag_13',
 'Sales_roll_mean_5_lag_13',
 'Sales_ewm_alpha_085_lag_13',
 'Sales_roll_mean_4_lag_13',
 'Sales_ewm_alpha_08_lag_14',
 'Product',
 'Sales_roll_median_5_lag_13',
 'weekly_inventory_category',
 'Sales_roll_min_6_lag_13',
 'Month',
 'Price_roll_max_5_lag_13',
 'Sales_roll_max_5_lag_13',
 'Sales_roll_max_4_lag_13',
 'Price_ewm_alpha_08_lag_14',
 'Price_roll_mean_3_lag_13',
 'Sales_roll_median_4_lag_13',
 'Price_roll_max_6_lag_13',
 'Sales_roll_min_5_lag_13',
 'Month_cos',
 'new_combination']

In [27]:
os.makedirs("outputs", exist_ok=True)
feats_df.to_pickle(os.path.join(os.getcwd(), "outputs", "feat_imp_all_feats.pkl"))

In [28]:
y_pred = predictor.predict(test_all_feats_df)
print("Predictions:  ", list(y_pred)[:5])

Predictions:   [4.025369644165039, 3.9795236587524414, 4.019855499267578, 4.0163373947143555, 4.559609889984131]


In [37]:
test_all_feats_df_op = test_all_feats_df[["Client", "Warehouse", "Product"]].copy()
test_all_feats_df_op["Sales"] = y_pred
test_all_feats_df_op = test_all_feats_df_op.reset_index()
test_all_feats_df_op['Date'] = test_all_feats_df_op['Date'].dt.strftime('%Y-%m-%d')

In [38]:
test_all_feats_df_op = test_all_feats_df_op.pivot(
    index=["Client", "Warehouse", "Product"], columns="Date", values="Sales"
).reset_index()

In [46]:
test_all_feats_df_op.index.name = ''

In [49]:
test_all_feats_df_op.shape

(15053, 16)

In [50]:
test_all_feats_df_op.to_csv("outputs/test_all_feats_df_op.csv", index=False)

###################################################### END ##############################################################